# Transactional SQL

## 1. Introduction

### What is a Transaction?
A transaction in SQL is a sequence of one or more SQL statements that are executed as a single unit of work. Transactions ensure that the database remains in a consistent state, even in the presence of failures or errors. 

---

### Why Are Transactions Important?
Transactions are essential in database management systems because they:
- Maintain **data integrity**.
- Ensure **consistency** of the database.
- Allow **safe concurrent access** by multiple users.
- Enable recovery from unexpected failures.

---

### The ACID Properties
Transactions are guided by four key properties collectively known as **ACID**:

1. **Atomicity**  
   A transaction is all-or-nothing. If any part of the transaction fails, the entire transaction is rolled back, leaving the database unchanged.

2. **Consistency**  
   A transaction ensures that the database transitions from one valid state to another, maintaining the integrity of the data.

3. **Isolation**  
    Transactions running concurrently operate as if they are the only ones executing in the system, ensuring they do not interfere with each other.

4. **Durability**  
   Once a transaction is committed, the changes it made are permanent, even in the event of a system crash.

---

### Real-World Example

Imagine a bank transfer:  
Alice wants to transfer \$500 to Bob. The system must:

1. Deduct \$500 from Alice's account.  
2. Add \$500 to Bob's account.  
3. Ensure both steps succeed or neither happens (to avoid money disappearing or being created).



This is a perfect use case for transactions! 

In [1]:
# Imports
from tabulate import tabulate
import psycopg2
import time

In [2]:
# Database connection parameters
dbname = 'banking_db'
user = 'postgres'
password = 'postgres'
host = 'postgres_db'  # This should be the service name defined in docker-compose.yml
port = '5432'  

In [3]:
# Establish connection to the PostgreSQL database
conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)

### Cursor in SQL

In SQL, a **cursor** is a database object that allows you to interact with the result set of a query in a controlled, row-by-row manner. It provides a pointer to a specific row within the result set, allowing you to process the data incrementally.

---

#### **Key Points of Cursors**:

- **Purpose**: A cursor enables you to retrieve rows from the database one by one or in chunks, and perform operations like updating, inserting, or deleting specific rows as needed.

- **How it Works**:
    - When you execute a query, the result set is generated by the database. 
    - The cursor helps navigate and interact with this result set.
    - You can use the cursor to move through the data row by row or in batches.

- **Common Operations**:
    - **Fetch Rows**: Cursors allow you to fetch one or more rows from the result set.
    - **Iterate Over Rows**: You can iterate through the result set one row at a time, making it easy to process large sets of data incrementally.
    - **Modify Data**: Cursors can be used to update, delete, or insert rows in a transaction.

---

In [4]:
# Create a cursor object to interact with the database
cur = conn.cursor()

In [5]:
# Query to fetch records from the accounts table
cur.execute("SELECT * FROM accounts")

# Fetch all records from the result
records = cur.fetchall()

# Fetch column names
col_names = [desc[0] for desc in cur.description]

# Print the records in table format
print(tabulate(records, headers=col_names, tablefmt="grid"))

+------+----------------+---------------------+----------+-----------+----------------------------+
|   id | name           | email               | phone    |   balance | created_at                 |
+======+================+=====================+==========+===========+============================+
|    3 | Charlie Brown  | charlie@example.com | 555-9876 |     10000 | 2025-03-27 23:25:31.364953 |
+------+----------------+---------------------+----------+-----------+----------------------------+
|    4 | David Williams | david@example.com   | 555-6543 |      2500 | 2025-03-27 23:25:31.364953 |
+------+----------------+---------------------+----------+-----------+----------------------------+
|    5 | Eve Davis      | eve@example.com     | 555-1122 |      3000 | 2025-03-27 23:25:31.364953 |
+------+----------------+---------------------+----------+-----------+----------------------------+
|    1 | Alice Smith    | alice@example.com   | 555-1234 |      4951 | 2025-03-27 23:25:31.364953 |


## 2. SQL Transactions

### Transaction Lifecycle

1. **BEGIN TRANSACTION**:
   
   Marks the start of a transaction. Any changes made after this command are part of the transaction and are not permanent until the transaction is committed.

2. **COMMIT**

   Applies all changes made during the transaction to the database. Once committed, the changes are permanent and cannot be rolled back.

3. **ROLLBACK**
   
   Undoes all changes made during the transaction, restoring the database to its state before the transaction began. This command is used if there is an error or if the transaction needs to be discarded.

---

### SQL Transaction Commands

1. **BEGIN TRANSACTION** (or **START TRANSACTION**)
   
   Starts a new transaction. Any changes made after this command are not permanent until the transaction is committed. This command signals the beginning of a unit of work.

2. **COMMIT**
   
   Finalizes the transaction and applies all changes to the database. Once committed, the changes become permanent and are visible to other transactions.

3. **ROLLBACK**
   
   Reverses all changes made since the transaction began, undoing any changes made during the transaction. This command is typically used when an error occurs or when a transaction should not be finalized.



## 2.1. A Trivial Transaction

In [6]:
# Query to fetch records from the accounts table to see tuples before the transaction
cur.execute("SELECT * FROM accounts WHERE name = 'Alice Smith' or name = 'Bob Johnson'")

# Fetch all records from the result
records = cur.fetchall()

# Print the records in table format
print(tabulate(records, headers=col_names, tablefmt="grid"))

+------+-------------+-------------------+----------+-----------+----------------------------+
|   id | name        | email             | phone    |   balance | created_at                 |
+======+=============+===================+==========+===========+============================+
|    1 | Alice Smith | alice@example.com | 555-1234 |      4951 | 2025-03-27 23:25:31.364953 |
+------+-------------+-------------------+----------+-----------+----------------------------+
|    2 | Bob Johnson | bob@example.com   | 555-5678 |      1549 | 2025-03-27 23:25:31.364953 |
+------+-------------+-------------------+----------+-----------+----------------------------+


In [7]:
# Start the transaction
cur.execute("BEGIN TRANSACTION;")

# Deduct $49 from Alice's account
cur.execute("UPDATE accounts SET balance = balance - 49 WHERE name = 'Alice Smith'")

# Add $49 to Bob's account
cur.execute("UPDATE accounts SET balance = balance + 49 WHERE name = 'Bob Johnson'")

# Commit the transaction to make changes permanent
cur.execute("COMMIT;")

In [8]:
# Query to fetch records from the accounts table to check if changes are reflected to the database
cur.execute("SELECT * FROM accounts WHERE name = 'Alice Smith' or name = 'Bob Johnson'")

# Fetch all records from the result
records = cur.fetchall()

# Print the records in table format
print(tabulate(records, headers=col_names, tablefmt="grid"))

+------+-------------+-------------------+----------+-----------+----------------------------+
|   id | name        | email             | phone    |   balance | created_at                 |
+======+=============+===================+==========+===========+============================+
|    1 | Alice Smith | alice@example.com | 555-1234 |      4902 | 2025-03-27 23:25:31.364953 |
+------+-------------+-------------------+----------+-----------+----------------------------+
|    2 | Bob Johnson | bob@example.com   | 555-5678 |      1598 | 2025-03-27 23:25:31.364953 |
+------+-------------+-------------------+----------+-----------+----------------------------+


## 2.1. Rolling Back a Transaction

#### Charlie Brown's account should not go below `$7000`, as he has a pending payment of `$7000` that must be accounted for.


In [9]:
# Query to fetch records from the accounts table to see tuples before the transaction
cur.execute("SELECT * FROM accounts WHERE name = 'Charlie Brown'")

# Fetch all records from the result
records = cur.fetchall()

# Print the records in table format
print(tabulate(records, headers=col_names, tablefmt="grid"))

+------+---------------+---------------------+----------+-----------+----------------------------+
|   id | name          | email               | phone    |   balance | created_at                 |
+======+===============+=====================+==========+===========+============================+
|    3 | Charlie Brown | charlie@example.com | 555-9876 |     10000 | 2025-03-27 23:25:31.364953 |
+------+---------------+---------------------+----------+-----------+----------------------------+


In [10]:
# Start the transaction
cur.execute("BEGIN TRANSACTION;")

# Deduct 3001 dollars from Chris's account (this is the risky operation)
cur.execute("UPDATE accounts SET balance = balance - 3001 WHERE name = 'Charlie Brown'")

# Check Chris's new balance
cur.execute("SELECT balance FROM accounts WHERE name = 'Charlie Brown'")
chris_balance = cur.fetchone()[0]

# If Chris's balance falls below 7000 dollars (due to the deduction), rollback the transaction
if chris_balance < 7000:
    print("Error: Chris's balance cannot go below 7000 dollars. Rolling back the transaction.")
    cur.execute("ROLLBACK")  # Undo all changes made during the transaction
else:
    # Commit the transaction to make changes permanent
    cur.execute("COMMIT")
    print("Transaction successful. Changes have been committed.")

Error: Chris's balance cannot go below 7000 dollars. Rolling back the transaction.


In [11]:
# Query to fetch records from the accounts table to check if changes are reflected to the database
cur.execute("SELECT * FROM accounts WHERE name = 'Charlie Brown'")

# Fetch all records from the result
records = cur.fetchall()

# Print the records in table format
print(tabulate(records, headers=col_names, tablefmt="grid"))

+------+---------------+---------------------+----------+-----------+----------------------------+
|   id | name          | email               | phone    |   balance | created_at                 |
+======+===============+=====================+==========+===========+============================+
|    3 | Charlie Brown | charlie@example.com | 555-9876 |     10000 | 2025-03-27 23:25:31.364953 |
+------+---------------+---------------------+----------+-----------+----------------------------+


## 2.2. An Aborted Transaction

In [12]:
# Query to fetch records from the accounts table to see tuples before the transaction
cur.execute("SELECT * FROM accounts WHERE name = 'Alice Smith' or name = 'Bob Johnson'")

# Fetch all records from the result
records = cur.fetchall()

# Print the records in table format
print(tabulate(records, headers=col_names, tablefmt="grid"))

+------+-------------+-------------------+----------+-----------+----------------------------+
|   id | name        | email             | phone    |   balance | created_at                 |
+======+=============+===================+==========+===========+============================+
|    1 | Alice Smith | alice@example.com | 555-1234 |      4902 | 2025-03-27 23:25:31.364953 |
+------+-------------+-------------------+----------+-----------+----------------------------+
|    2 | Bob Johnson | bob@example.com   | 555-5678 |      1598 | 2025-03-27 23:25:31.364953 |
+------+-------------+-------------------+----------+-----------+----------------------------+


In [13]:
# Start the transaction
cur.execute("BEGIN TRANSACTION;")

try:
    # Deduct 2500 dollars from Bob's account (this will raise an error if Bob doesn't have enough funds)
    cur.execute("UPDATE accounts SET balance = balance - 2500 WHERE name = 'Bob Johnson'")

    # Add 2500 dollars to Alice's account
    cur.execute("UPDATE accounts SET balance = balance + 2500 WHERE name = 'Alice Smith'")

    # Commit the transaction to make changes permanent
    cur.execute("COMMIT;")
    print("Transaction successful: 2500 dollars transferred from Bob Johnson to Alice Smith.")

except Exception as e:
    # If an error occurs (like insufficient funds), rollback the transaction to undo all changes
    print(f"Error: {e}")
    cur.execute("ROLLBACK;")


Error: new row for relation "accounts" violates check constraint "accounts_balance_check"
DETAIL:  Failing row contains (2, Bob Johnson, bob@example.com, 555-5678, -902.00, 2025-03-27 23:25:31.364953).



In [14]:
# Query to fetch records from the accounts table to check if changes are reflected to the database
cur.execute("SELECT * FROM accounts WHERE name = 'Alice Smith' or name = 'Bob Johnson'")

# Fetch all records from the result
records = cur.fetchall()

# Print the records in table format
print(tabulate(records, headers=col_names, tablefmt="grid"))

+------+-------------+-------------------+----------+-----------+----------------------------+
|   id | name        | email             | phone    |   balance | created_at                 |
+======+=============+===================+==========+===========+============================+
|    1 | Alice Smith | alice@example.com | 555-1234 |      4902 | 2025-03-27 23:25:31.364953 |
+------+-------------+-------------------+----------+-----------+----------------------------+
|    2 | Bob Johnson | bob@example.com   | 555-5678 |      1598 | 2025-03-27 23:25:31.364953 |
+------+-------------+-------------------+----------+-----------+----------------------------+


## 2.3. System Failure during Execution

In [15]:
# Query to fetch records from the accounts table to see tuples before the transaction
cur.execute("SELECT * FROM accounts WHERE name = 'Alice Smith' or name = 'Bob Johnson'")

# Fetch all records from the result
records = cur.fetchall()

# Print the records in table format
print(tabulate(records, headers=col_names, tablefmt="grid"))

+------+-------------+-------------------+----------+-----------+----------------------------+
|   id | name        | email             | phone    |   balance | created_at                 |
+======+=============+===================+==========+===========+============================+
|    1 | Alice Smith | alice@example.com | 555-1234 |      4902 | 2025-03-27 23:25:31.364953 |
+------+-------------+-------------------+----------+-----------+----------------------------+
|    2 | Bob Johnson | bob@example.com   | 555-5678 |      1598 | 2025-03-27 23:25:31.364953 |
+------+-------------+-------------------+----------+-----------+----------------------------+


In [16]:
# Start the transaction
cur.execute("BEGIN TRANSACTION;")

# Deduct 1000 dollars from Bob's account (this will be rolled back if power loss occurs)
cur.execute("UPDATE accounts SET balance = balance - 1000 WHERE name = 'Bob Johnson'")

# Add 1000 dollars to Alice's account
cur.execute("UPDATE accounts SET balance = balance + 1000 WHERE name = 'Alice Smith'")

# Simulate power loss by disconnecting the database
print("Simulating power loss... Disconnecting from the database.")
conn.close()

# Wait for a few seconds (simulate time of power loss)
time.sleep(3)

Simulating power loss... Disconnecting from the database.


In [17]:
# Assume now, the database has been recovered from the failure and we can reconnect
conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)
cur = conn.cursor()

# Query to fetch records from the accounts table to check if changes are reflected to the database
cur.execute("SELECT * FROM accounts WHERE name = 'Alice Smith' or name = 'Bob Johnson'")

# Fetch all records from the result
records = cur.fetchall()

# Print the records in table format
print(tabulate(records, headers=col_names, tablefmt="grid"))

+------+-------------+-------------------+----------+-----------+----------------------------+
|   id | name        | email             | phone    |   balance | created_at                 |
+======+=============+===================+==========+===========+============================+
|    1 | Alice Smith | alice@example.com | 555-1234 |      4902 | 2025-03-27 23:25:31.364953 |
+------+-------------+-------------------+----------+-----------+----------------------------+
|    2 | Bob Johnson | bob@example.com   | 555-5678 |      1598 | 2025-03-27 23:25:31.364953 |
+------+-------------+-------------------+----------+-----------+----------------------------+


In [18]:
# Close the connection
cur.close()
conn.close()

## 3. Isolation Levels in SQL

## 4. CAP Theorem
